In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from ipywidgets import interact
from scipy.signal import savgol_filter
import numpy as np

from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import quicklook_xarray
from gfatpy.utils.utils import moving_average

from gfatpy.atmo.ecmwf import get_ecmwf_temperature_pressure
from gfatpy.atmo.freudenthaler_molecular_properties import molecular_properties

%load_ext autoreload
%autoreload 2

%matplotlib inline

### Testing

Performance is tested with preprocessing `mhc_1a_Prs_rs_xf_20210705.nc`, with all the default flags and gluing products with only the channels `["532xpa", "532xpp"]`.

Timing is carrie on by the `%%time` at the beggining of the cell.

In [ ]:
lidar_ng = preprocess(
    "mhc_1a_Prs_rs_xf_20210626.nc",
    channels = ["532xpa", "532xpp"],
    # gluing_products=True
)

an = (lidar_ng.signal_532xpa * lidar_ng.range ** 2).values
pc = (lidar_ng.signal_532xpp * lidar_ng.range ** 2).values

In [ ]:
w_size = 70
prof_idx = 7

w1 = np.lib.stride_tricks.sliding_window_view(an[prof_idx], w_size)
w2 = np.lib.stride_tricks.sliding_window_view(pc[prof_idx], w_size)

corrcoefs = np.array([])

for idx in range(w1.shape[0]):
    
    _w1 = w1[idx]
    _w2 = w2[idx]
    coeff = np.corrcoef(_w1, _w2)[1, 0]
    corrcoefs = np.hstack([corrcoefs, coeff])

In [ ]:
w_size = 70

w1 = np.lib.stride_tricks.sliding_window_view(np.gradient(an[prof_idx]), w_size)
w2 = np.lib.stride_tricks.sliding_window_view(np.gradient(pc[prof_idx]), w_size)

corrcoefs_diff = np.array([])

for idx in range(w1.shape[0]):
    
    _w1 = w1[idx]
    _w2 = w2[idx]
    coeff = np.corrcoef(_w1, _w2)[1, 0]
    corrcoefs_diff = np.hstack([corrcoefs_diff, coeff])

In [ ]:
plt.plot(corrcoefs)
plt.plot(corrcoefs_diff)

In [ ]:
import numba

In [ ]:
def corrcoefs(arr1, arr2):
    w1 = np.lib.stride_tricks.sliding_window_view(arr1, w_size)
    w2 = np.lib.stride_tricks.sliding_window_view(arr2, w_size)
    _corrcoefs = np.zeros(w1.shape[0])
    for idx in range(w1.shape[0]): 
        _w1 = w1[idx]
        _w2 = w2[idx]
        coeff = np.corrcoef(_w1, _w2)[1, 0]
        _corrcoefs[idx] = coeff
    return _corrcoefs

@numba.njit()
def _rolling(a, window):
    shape = (a.size - window + 1, window)
    strides = (a.itemsize, a.itemsize)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


@numba.njit(parallel=True)
def windowed_corrcoefs(arr1: np.ndarray, arr2: np.ndarray, w_size: int):
    range_shape = arr1.shape[1] - (w_size - 1)
    _corrcoefs = np.zeros((arr1.shape[0], range_shape))
    for t_idx in numba.prange(arr1.shape[0]):
        w1 = _rolling(arr1[t_idx], w_size)
        w2 = _rolling(arr2[t_idx], w_size)
        for idx in numba.prange(w1.shape[0]):
            _w1 = w1[idx]
            _w2 = w2[idx]
            coeff = np.corrcoef(_w1, _w2)[1, 0]
            _corrcoefs[t_idx][idx] = coeff
    return _corrcoefs


In [ ]:
180*7.5

In [ ]:
corr_coeffs = windowed_corrcoefs(an[:, 0:750], pc[:, 0:750] , 140)

In [ ]:
plt.plot(corr_coeffs[0])

In [ ]:
@interact(idx=(0, corr_coeffs.shape[0] -1, 1))
def correlation(idx):
    plt.plot(corr_coeffs[idx])

In [ ]:
for i in corr_coeffs:
    plt.plot(i, lw=0.05, c = "k")

In [ ]:
(corr_coeffs[511].argmax() + 70) * 7.5

In [ ]:
plt.plot(
    np.apply_along_axis(lambda x: x.argmax(), 1,corr_coeffs),
    c = 'r',
    lw = 0,
    marker = '.'
)

plt.plot(savgol_filter(
    np.apply_along_axis(lambda x: x.argmax(), 1,corr_coeffs),
    11,
    2
    ),
    lw = 0,
    marker = '.',
    c = 'g'
)

In [ ]:
%%time
corrcoefs(an[0], pc[0])

In [ ]:
moving_average(np.array([np.apply_along_axis(lambda x: x.argmax(), 1,corr_coeffs)]))

In [ ]:
lidar = preprocess(
    "../tests/datos/MULHACEN/1a/2021/07/05/mhc_1a_Prs_rs_xf_20210705.nc",
    channels = ["532xpa", "532xpp"],
    gluing_products=True
)

In [ ]:
%matplotlib notebook
quicklook_xarray(lidar.signal_532xpg, is_rcs = False)

In [ ]:
quicklook_xarray(lidar.signal_532xpa, is_rcs = False)

In [ ]:
quicklook_xarray(lidar.signal_532xpp, is_rcs = False)

In [ ]:
%matplotlib inline


@interact(idx=(0, lidar.time.shape[0] - 1, 1))
def plot_profile(idx):
    plt.figure(figsize=(15, 7))
    plt.plot(lidar.range, lidar.signal_532xpg[idx] * lidar.range ** 2, label = "GL")
    plt.plot(lidar.range, lidar.signal_532xpp[idx] * lidar.range ** 2, label = "PC", lw = 0.6)
    plt.xlim(0, 5000)
    plt.legend()
    
    plt.plot(lidar.range, 
        (lidar.signal_532xpa[idx] * 
         lidar.signal_532xpp[idx].sel(range=4000, method="nearest") / 
         lidar.signal_532xpa[idx].sel(range=4000, method="nearest")
    )  * lidar.range ** 2, 
        label = "AN", 
        lw = 0.6)
    
    plt.xlim(0, 5000)
    plt.legend()

In [ ]:
plt.figure(figsize=(15, 7))

plt.plot(lidar.range, lidar.signal_532xpg[214] * lidar.range ** 2)
plt.plot(lidar.range, lidar.signal_532xpg[215] * lidar.range ** 2)
plt.plot(lidar.range, lidar.signal_532xpg[216] * lidar.range ** 2)
plt.plot(lidar.range, lidar.signal_532xpg[217] * lidar.range ** 2, c = 'k')
plt.plot(lidar.range, lidar.signal_532xpg[218] * lidar.range ** 2)
plt.plot(lidar.range, lidar.signal_532xpg[219] * lidar.range ** 2)
#plt.plot(lidar.range, lidar.signal_532xpg[217] * lidar.range ** 2)
plt.xlim(0, 5000)
plt.ylim(1e7, 1e9)

plt.yscale("log")
plt.legend()

In [ ]:
lidar.time[217]